In [1]:
# import cv2


# # Skeleton base for starting a webcam and keeping it alive until key interrupt
# # is given
# cap = cv2.VideoCapture(0) # 0 for default camera of the machine (PC)

# while True:
#     _, frame = cap.read()
#     frame = cv2.flip(frame, 1) # mirror the webcam (1 - flip vertically)

#     greyScale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     cv2.imshow("Drowsiness Detection System", frame)
#     key = cv2.waitKey(10) # returns the ASCII value of the key pressed

#     # Esc, 'Q', or 'q' exits the webcam
#     # ord() returns the ASCII value of the parameter
#     if key in [27, ord('q'), ord('Q')]:
#         break
# # end while

# cap.release()
# cv2.destroyAllWindows()

In [2]:
import dlib

# Face detection or mapping face to get eyes
faceDetector = dlib.get_frontal_face_detector()

# Put the location of .DAT file for predicting the landmarks on face
pathToDotDATFile = "shape_predictor_68_face_landmarks\ \
    shape_predictor_68_face_landmarks.dat"
pathToDotDATFile = "".join(pathToDotDATFile.split())

dlibFaceLandmark = dlib.shape_predictor(pathToDotDATFile)

### Face Landmark Mapping Points
The image below shows all the masking and landmarks numerically.

![Face Mask Image](images\facemask.png "Face Mask Image")

The landmarks are numbered from 0-67. The landmarks are mapped as follows:
- 0-16: Jawline
- 17-21: Right Eyebrow
- 22-26: Left Eyebrow
- 27-30: Nose Bridge
- 30-35: Lower Nose
- 36-41: Right Eye
- 42-47: Left Eye
- 48-60: Outer Lip
- 61-67: Inner Lip

### Needed Points for Project - Points of Both the Eyes
The image below is in reference to the previous image. We are here focusing on the points of the eyes. We will be using these points to calculate the eye aspect ratio.

![Eye Mask Image](images/eyesmask.png "Eye Mask Image")


##### Eye's Aspect Ratio Calculation:
We shall write a function `calculateAspectRatioOfEye(eye)` to calculate the eye's aspect ratio. The eye's aspect ratio is the ratio of the vertical distance between the points of the eye and the horizontal distance between the extreme points of the eye.

In the function `calculateAspectRatioOfEye(eye)`, the parameter `eye` is a list of 6 points of the eye. For the right eye, the list would be `[36, 37, 38, 39, 40, 41]` and for the left eye, the list would be `[42, 43, 44, 45, 46, 47]`.

Vertical Distance between the points of the right eye would be the average of the distance between the points - `(37, 41)` and `(38, 40)`. Similarly, the vertical distance between the points of the left eye would be the average of the distance between the points - `(43, 47)` and `(44, 46)`.

Horizontal Distance is comparatively simpler to calulate: It is the distance between the extreme points of the eye. For the right eye, it is the distance between the points `(36, 39)` and for the left eye, it is the distance between the points `(42, 45)`.

The aspect ratio of the eye is the ratio of the vertical distance (which is the average of the two vertical distances calculated) and the horizontal distance.

The function returns the aspect ratio of the eye.

In [3]:
from scipy.spatial import distance

# "eye" is a list of points that correspond to the eye landmarks
def calculateAspectRatioOfEye(eye: list) -> float:
    """
    Calculate the aspect ratio of an eye based on the distance between the top
    and bottom of the eye and the distance between the left and right of
    the eye
    """
    eyeHeight_leftPart = distance.euclidean(eye[1], eye[5])
    eyeHeight_rightPart = distance.euclidean(eye[2], eye[4])
    eyeHeight = (eyeHeight_leftPart + eyeHeight_rightPart) / 2
    
    eyeWidth = distance.euclidean(eye[0], eye[3])
    
    return (eyeHeight / eyeWidth)
# end function calculateAspectRatioOfEye()

##### Extracting the eye landmarks from the image and drawing the eye region bounding lines

In [4]:
# import cv2


# cap = cv2.VideoCapture(0) # 0 for default camera of the machine (PC)

# while True:
#     _, frame = cap.read()
#     frame = cv2.flip(frame, 1) # mirror the webcam (1 - flip vertically)

#     # Colour image not required for drowsiness detection (face detection)
#     greyScaleImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Detect face using dlib's faceDetector
#     faces = faceDetector(greyScaleImg) # list of rectangle coordinates of face

#     # Iterate over the faces detected and get the eyes' landmarks
#     for face in faces:
#         faceLandmarks = dlibFaceLandmark(greyScaleImg, face)
#         leftEye, rightEye = [], []

#         # Get the landmarks of the right eye
#         for n in range(36, 42):
#             x_Coordinate = faceLandmarks.part(n).x
#             y_Coordinate = faceLandmarks.part(n).y
#             rightEye.append((x_Coordinate, y_Coordinate))

#             # Draw a line surrounding the eye
#             ## For that we need the next coordinates as well
#             #### if n == 41, nextPoint = 36 -> to make a loop around the eye
#             if n == 41:
#                 nextPoint = 36
#             else:
#                 nextPoint = n + 1
#             # end if-else

#             x2_Coordinate = faceLandmarks.part(nextPoint).x
#             y2_Coordinate = faceLandmarks.part(nextPoint).y
#             cv2.line(frame, pt1=(x_Coordinate, y_Coordinate),
#                      pt2=(x2_Coordinate, y2_Coordinate),
#                      color=(0, 255, 0), thickness=1)
#         # end for
        
#         # Get the landmarks of the left eye
#         for n in range(42, 48):
#             x_Coordinate = faceLandmarks.part(n).x
#             y_Coordinate = faceLandmarks.part(n).y
#             leftEye.append((x_Coordinate, y_Coordinate))

#             # Draw a line surrounding the eye
#             if n == 47:
#                 nextPoint = 42
#             else:
#                 nextPoint = n + 1
            
#             x2_Coordinate = faceLandmarks.part(nextPoint).x
#             y2_Coordinate = faceLandmarks.part(nextPoint).y
#             cv2.line(frame, (x_Coordinate, y_Coordinate),
#                      (x2_Coordinate, y2_Coordinate),
#                      (0, 255, 255), 1)
#         # end for

#     cv2.imshow("Drowsiness Detection System", frame)
#     key = cv2.waitKey(10) # returns the ASCII value of the key pressed

#     # Esc, 'Q', or 'q' exits the webcam
#     # ord() returns the ASCII value of the parameter
#     if key in [27, ord('q'), ord('Q')]:
#         break
# # end while

# cap.release()
# cv2.destroyAllWindows()

##### Final Code for the Drowsiness Detection Project
We shall calculate the aspect ratio of each eye and take its average and if the average is less than the threshold value (_here, we shall consider the threshold as 0.25_), we shall consider that the person is drowsy and sound an alarm.

For the alarm we shall use the `pyttsx3` library which is a text-to-speech conversion library in Python. It works offline, and is compatible with both Python 2 and 3.

In [5]:
import pyttsx3

# Initialising the pyttsx3 engine so that we can use it we can deliver an audio
# message to the user if they are drowsy
engine = pyttsx3.init()

In [6]:
# import cv2


# cap = cv2.VideoCapture(0) # 0 for default camera of the machine (PC)
# cap.set(3, 640) # 3 -> CAP_PROP_FRAME_WIDTH
# cap.set(4, 480) # 4 -> CAP_PROP_FRAME_HEIGHT

# # Run webcam until 'Esc', 'Q', or 'q' is pressed
# while True:
#     _, frame = cap.read()
#     frame = cv2.flip(frame, 1) # mirror the webcam (1 - flip vertically)

#     # Colour image not required for drowsiness detection (face detection)
#     greyScaleImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Detect face using dlib's faceDetector
#     faces = faceDetector(greyScaleImg) # list of rectangle coordinates of face

#     # Iterate over the faces detected and get the eyes' landmarks
#     for face in faces:
#         faceLandmarks = dlibFaceLandmark(greyScaleImg, face)
#         leftEye, rightEye = [], []

#         # Get the landmarks of the right eye
#         for n in range(36, 42):
#             x_Coordinate = faceLandmarks.part(n).x
#             y_Coordinate = faceLandmarks.part(n).y
#             rightEye.append((x_Coordinate, y_Coordinate))

#             # Draw a line surrounding the eye
#             ## For that we need the next coordinates as well
#             #### if n == 41, nextPoint = 36 -> to make a loop around the eye
#             if n == 41:
#                 nextPoint = 36
#             else:
#                 nextPoint = n + 1
#             # end if-else

#             x2_Coordinate = faceLandmarks.part(nextPoint).x
#             y2_Coordinate = faceLandmarks.part(nextPoint).y
#             cv2.line(frame, pt1=(x_Coordinate, y_Coordinate),
#                      pt2=(x2_Coordinate, y2_Coordinate),
#                      color=(0, 255, 0), thickness=1)
#         # end for
        
#         # Get the landmarks of the left eye
#         for n in range(42, 48):
#             x_Coordinate = faceLandmarks.part(n).x
#             y_Coordinate = faceLandmarks.part(n).y
#             leftEye.append((x_Coordinate, y_Coordinate))

#             # Draw a line surrounding the eye
#             if n == 47:
#                 nextPoint = 42
#             else:
#                 nextPoint = n + 1
            
#             x2_Coordinate = faceLandmarks.part(nextPoint).x
#             y2_Coordinate = faceLandmarks.part(nextPoint).y
#             cv2.line(frame, (x_Coordinate, y_Coordinate),
#                      (x2_Coordinate, y2_Coordinate),
#                      (0, 255, 255), 1)
#         # end for

#         # Calculate the aspect ratio (AR) for both eyes and take average
#         rightEyeAR = calculateAspectRatioOfEye(rightEye)
#         leftEyeAR = calculateAspectRatioOfEye(leftEye)

#         averageEyeAR = (rightEyeAR + leftEyeAR) / 2.0
#         averageEyeAR = round(averageEyeAR, 2) # round of average to 2 decimal
#                                               # places

#         # Check against the threshold value for drowsiness
#         if averageEyeAR < 0.25:
#             cv2.putText(frame, text="Drowsy!!!", org=(10, 30),
#                         fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2,
#                         color=(0, 0, 255), thickness=2)
#             # Also call audio engine for alarm
#             engine.say("Wake up, Sleepy Head!!!!!!", "wake-up-alert")
#             engine.runAndWait()
#         # end if

#     cv2.imshow("Drowsiness Detection System", frame)
#     key = cv2.waitKey(10) # returns the ASCII value of the key pressed

#     # Esc, 'Q', or 'q' exits the webcam
#     # ord() returns the ASCII value of the parameter
#     if key in [27, ord('q'), ord('Q')]:
#         break
#     # end if
# # end while

# cap.release()
# cv2.destroyAllWindows()

# # end of program

This is working fine and great. But there's one issue we can try to solve here - the program sounds the alarm even if it catches the user merely blinking. One way to fix this is to maintain a counter of frames and if the eyes are closed for a threshold number of frames, we can assume that the person has fallen asleep and then sound the alarm. This threshold can be set depending on the user or the application. In this case, I have set it to 20, i.e., if the eyes remain closed for 20 consecutive frames, we assume that the person has fallen asleep.

In [7]:
import cv2


cap = cv2.VideoCapture(0) # 0 for default camera of the machine (PC)
cap.set(3, 640) # 3 -> CAP_PROP_FRAME_WIDTH
cap.set(4, 480) # 4 -> CAP_PROP_FRAME_HEIGHT

frameCount = 0

# Run webcam until 'Esc', 'Q', or 'q' is pressed
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1) # mirror the webcam (1 - flip vertically)

    # Colour image not required for drowsiness detection (face detection)
    greyScaleImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face using dlib's faceDetector
    faces = faceDetector(greyScaleImg) # list of rectangle coordinates of face

    # Iterate over the faces detected and get the eyes' landmarks
    for face in faces:
        faceLandmarks = dlibFaceLandmark(greyScaleImg, face)
        leftEye, rightEye = [], []

        # Get the landmarks of the right eye
        for n in range(36, 42):
            x_Coordinate = faceLandmarks.part(n).x
            y_Coordinate = faceLandmarks.part(n).y
            rightEye.append((x_Coordinate, y_Coordinate))

            # Draw a line surrounding the eye
            ## For that we need the next coordinates as well
            #### if n == 41, nextPoint = 36 -> to make a loop around the eye
            if n == 41:
                nextPoint = 36
            else:
                nextPoint = n + 1
            # end if-else

            x2_Coordinate = faceLandmarks.part(nextPoint).x
            y2_Coordinate = faceLandmarks.part(nextPoint).y
            cv2.line(frame, pt1=(x_Coordinate, y_Coordinate),
                     pt2=(x2_Coordinate, y2_Coordinate),
                     color=(0, 255, 0), thickness=1)
        # end for
        
        # Get the landmarks of the left eye
        for n in range(42, 48):
            x_Coordinate = faceLandmarks.part(n).x
            y_Coordinate = faceLandmarks.part(n).y
            leftEye.append((x_Coordinate, y_Coordinate))

            # Draw a line surrounding the eye
            if n == 47:
                nextPoint = 42
            else:
                nextPoint = n + 1
            
            x2_Coordinate = faceLandmarks.part(nextPoint).x
            y2_Coordinate = faceLandmarks.part(nextPoint).y
            cv2.line(frame, (x_Coordinate, y_Coordinate),
                     (x2_Coordinate, y2_Coordinate),
                     (0, 255, 255), 1)
        # end for

        # Calculate the aspect ratio (AR) for both eyes and take average
        rightEyeAR = calculateAspectRatioOfEye(rightEye)
        leftEyeAR = calculateAspectRatioOfEye(leftEye)

        averageEyeAR = (rightEyeAR + leftEyeAR) / 2.0
        averageEyeAR = round(averageEyeAR, 2) # round of average to 2 decimal
                                              # places

        # Check against the threshold value for drowsiness
        if averageEyeAR < 0.25:
            # Check against the frame count to avoid false alarms like that 
            # when blinking
            if frameCount >= 20:
                cv2.putText(frame, text="Drowsy!!!", org=(100, 100),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2,
                            color=(0, 0, 255), thickness=2)
                # Also call audio engine for alarm
                engine.say("Wake up, Sleepy Head!!!!!!", "wake-up-alert")
                engine.runAndWait()
            else:
                frameCount += 1
            # end if-else
        else:
            # Keep the framecount at zero if the eyes are open
            frameCount = 0
        # end if-else


    cv2.imshow("Drowsiness Detection System", frame)
    key = cv2.waitKey(10) # returns the ASCII value of the key pressed

    # Esc, 'Q', or 'q' exits the webcam
    # ord() returns the ASCII value of the parameter
    if key in [27, ord('q'), ord('Q')]:
        break
    # end if
# end while

cap.release()
cv2.destroyAllWindows()

# end of program

Much better. Though there are some irregularities based on the distance from the camera.

Also, we don't need to detect the landmarks for every frame as it is an expensive operation. Instead, we can detect the landmarks after every n frames (I have used n=20 in this case) and hence saving a lot of computation power. This can be done as shown below: